# Simple Tool & LLM Integration Demo

Let's start with a very simple example for how tools work.  
In this case, we will create the tool here inside the notebook, so that you can see how it works, and then use it both manually and with an LLM.  
The tool we will test here is a simple calculator, and although it's simple it's quite effective when combined with LLMs as they often have trouble calculating.

## Creating the calculator tool

A tool is essentially a function that:
- Takes structured input (JSON)
- Performs a specific action
- Returns structured output (JSON)

In [ ]:
import json
from typing import Dict, Any

class CalculatorTool:
    """A simple calculator tool that performs basic arithmetic operations."""
    
    def __init__(self):
        self.name = "calculator"
        self.description = """Performs basic arithmetic operations (add, subtract, multiply, divide). 
Expected input format:
{
    "operation": "add" | "subtract" | "multiply" | "divide",
    "a": float,
    "b": float
}

Returns:
{
    "result": float
}
"""
    
    def execute(self, tool_input: Dict[str, Any]) -> Dict[str, Any]:
        """
        Execute the calculator tool.
        
        Expected input format:
        {
            "operation": "add" | "subtract" | "multiply" | "divide",
            "a": float,
            "b": float
        }
        
        Returns:
        {
            "result": float
        }
        """
        try:
            operation = tool_input.get("operation")
            a = float(tool_input.get("a", 0))
            b = float(tool_input.get("b", 0))
            
            if operation == "add":
                result = a + b
            elif operation == "subtract":
                result = a - b
            elif operation == "multiply":
                result = a * b
            elif operation == "divide":
                if b == 0:
                    return {
                        "error": "Division by zero is not allowed",
                    }
                result = a / b
            else:
                return {
                    "error": f"Unknown operation: {operation}",
                }
            
            return {
                "result": result,
            }
        except Exception as e:
            return {
                "error": str(e)
            }

# Create an instance of the tool
calculator = CalculatorTool()
print(f"Tool created: {calculator.name}")
print(f"Description: {calculator.description}")

## Using the Tool Directly

Now let's test our tool by sending a few JSON inputs and receiving JSON outputs:

In [ ]:
# Example 1: Addition
input_json = {
    "operation": "add",
    "a": 15,
    "b": 27
}

print("Input JSON:")
print(json.dumps(input_json, indent=2))

output = calculator.execute(input_json)

print("\nOutput JSON:")
print(json.dumps(output, indent=2))

In [ ]:
# Example 2: Multiplication
input_json = {
    "operation": "multiply",
    "a": 8,
    "b": 7
}

print("Input JSON:")
print(json.dumps(input_json, indent=2))

output = calculator.execute(input_json)

print("\nOutput JSON:")
print(json.dumps(output, indent=2))

In [ ]:
# Example 3: Division
input_json = {
    "operation": "divide",
    "a": 100,
    "b": 4
}

print("Input JSON:")
print(json.dumps(input_json, indent=2))

output = calculator.execute(input_json)

print("\nOutput JSON:")
print(json.dumps(output, indent=2))

Great! That's the basics of how a tool works, it's simply a service that accepts json input and responds with json output.  
But so far we are calling it ourselves, which is great and all but we can already use calculators, so how do we get the LLM to use it?

## LLM in the Loop

Now let's integrate an LLM to:
1. Understand a user's request
2. Format the request as proper JSON for the tool
3. Call the tool
4. Interpret the results for the user and respond

First, let's set up the Llama Stack client:

In [ ]:
!pip3 -q install llama-stack-client==0.3.0

In [ ]:
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger

# Connect to Llama Stack Server
base_url = "http://llama-stack-service:8321"
client = LlamaStackClient(base_url=base_url)
model = "llama32"

print(f"Connected to Llama Stack at: {base_url}")
print(f"Using model: {model}")

### LLM interprets user request and generates tool call

Notice how we can use the tool description in here when we craft our system prompt. This will be very useful later as we add more and more tools - we won't need to manually update the prompt, we just reference the tool's description!

### Step-by-Step Agent Walkthrough

Let's walk through each step individually so you can see the inputs and outputs clearly.

**Step 1: Define the user request**

In [ ]:
# This is what the user asks in natural language
user_request = "What is 45 plus 78?"

print("USER REQUEST:")
print(user_request)

**Step 2: Create a system prompt using the tool description**

In [ ]:
# Build the system prompt dynamically using the tool's description
system_prompt = f"""You are a helpful assistant that converts natural language requests into JSON tool calls.

Available tool: {calculator.name}
{calculator.description}

When the user asks you to perform a calculation, respond with ONLY a JSON object following the format above.
Do not include any explanation, just return the JSON object.
"""

print("SYSTEM PROMPT:")
print(system_prompt)

**Step 3: Send the request to the LLM to generate a tool call**

In [ ]:
# Ask the LLM to convert the natural language request into a tool call
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_request}
    ],
    stream=False,
)

# Extract the LLM's response
llm_response = response.choices[0].message.content

print("LLM RESPONSE:")
print(llm_response)

**Step 4: Parse the LLM response and send it to the tool**

In [ ]:
# Clean up the response (sometimes LLMs wrap JSON in markdown code blocks)
cleaned_response = llm_response.strip()
if cleaned_response.startswith("```"):
    # Remove markdown code block markers
    lines = cleaned_response.split("\n")
    cleaned_response = "\n".join(lines[1:-1])
    if cleaned_response.startswith("json"):
        cleaned_response = cleaned_response[4:].strip()

# Parse the JSON
tool_call = json.loads(cleaned_response)

tool_result = calculator.execute(tool_call)

print("TOOL RESULT:")
print(json.dumps(tool_result, indent=2))

**Step 5: Send the result back to the LLM for interpretation**

In [ ]:
# Ask the LLM to answer the user's question with the tool result
interpretation_prompt = f"""User question: {user_request}
Tool result: {json.dumps(tool_result)}

Answer the user's question directly with the result. Be concise."""

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Answer user questions directly and concisely."},
        {"role": "user", "content": interpretation_prompt}
    ],
    stream=True,
)

print("FINAL RESPONSE TO USER:")
print("-" * 60)
for log in EventLogger().log(response):
    log.print()
print("-" * 60)

We got a reply, great!  
As usual, feel free to change the prompts if you would prefer a different tone of the response.

### Complete Tool Workflow Function

Now that you've seen each step individually, let's wrap it all into a reusable function that does everything in one go!  
You can see that we call this function `run_agent`, this is because this is a very simple agentic workflow we have right now where it automatically calls the model multiple (2) times and uses a tool.  
I know.. it's not very autonomous yet, but hang in there, we are getting there ;)

In [ ]:
def run_agent(user_request: str):
    """
    Complete agent flow:
    1. User makes a natural language request
    2. LLM interprets and formats as tool call
    3. Tool executes
    4. LLM interprets results and responds to user
    """
    print("="*60)
    print(f"USER REQUEST: {user_request}")
    print("="*60)

    # Step 1: Build system prompt using tool description
    system_prompt = f"""You are a helpful assistant that converts natural language requests into JSON tool calls.

Available tool: {calculator.name}
{calculator.description}

When the user asks you to perform a calculation, respond with ONLY a JSON object following the format above.
Do not include any explanation, just return the JSON object.
"""

    # Step 2: LLM formats the tool call
    print("\n[Step 1] LLM interpreting request...")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_request}
        ],
        stream=False,
    )

    llm_response = response.choices[0].message.content

    # Step 3: Parse the response
    try:
        cleaned_response = llm_response.strip()
        if cleaned_response.startswith("```"):
            lines = cleaned_response.split("\n")
            cleaned_response = "\n".join(lines[1:-1])
            if cleaned_response.startswith("json"):
                cleaned_response = cleaned_response[4:].strip()

        tool_call = json.loads(cleaned_response)
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM response: {e}")
        print(f"LLM response was: {llm_response}")
        return

    print("\n[Step 2] Generated Tool Call:")
    print(json.dumps(tool_call, indent=2))

    # Step 4: Execute the tool
    print("\n[Step 3] Executing tool...")
    tool_result = calculator.execute(tool_call)
    print("\n[Step 4] Tool Result:")
    print(json.dumps(tool_result, indent=2))

    # Step 5: LLM interprets the result
    print("\n[Step 5] LLM interpreting results for user...")
    interpretation_prompt = f"""User question: {user_request}
Tool result: {json.dumps(tool_result)}

Answer the user's question directly with the result. Be concise."""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer user questions directly and concisely."},
            {"role": "user", "content": interpretation_prompt}
        ],
        stream=True,
    )

    print("\n[Step 6] FINAL RESPONSE TO USER:")
    print("-" * 60)
    for log in EventLogger().log(response):
        log.print()
    print("-" * 60)
    print()

### Let's try it! 🎉

Now we can make natural language requests and the LLM will:
1. Understand what we want
2. Format it as a proper tool call
3. Execute the tool
4. Explain the results

In [ ]:
# Example 1: Simple addition
run_agent("What is 45 plus 78?")

In [ ]:
# Example 2: Multiplication
run_agent("Can you multiply 12 by 15 for me?")

Let's try our error handling as well, what happens if we divide by 0 for example? 🔥

In [ ]:
# Example 3: Division
run_agent("I need to divide 144 by 0")

In [ ]:
# Example 4: Subtraction
run_agent("What's 100 minus 37?")

## Summary

In this notebook, we learned:

1. **Creating a Tool**: A tool is a simple class/function that:
   - Takes JSON input with specific parameters
   - Performs an action (calculation in our case)
   - Returns JSON output with results

2. **Using a Tool Directly**: We can call tools directly by providing properly formatted JSON

3. **LLM in the Loop**: The LLM acts as an intelligent interface that:
   - Interprets user requests
   - Formats them as proper tool calls
   - Interprets tool results for the user
   - Provides text responses

This pattern is the foundation of **AI Agents** - systems where LLMs can understand user intent, call appropriate tools, and provide meaningful responses! 🚀